# Home assignment 3

You should work on the assignement in groups of 2 participants. 

Upload your solution as a jupyter notebook to L2P by 17th of July 23:59h. (The deadline is strict)

Do not forget to specify the names of all contributing students in the jupyter notebook.

You should add comments to your code where necessary and print the relevant results.

#### Students: Xiaoqi Ma (383420), Anna Wegmann (330509)

# Network representation
Given a network obtain representations for the nodes in the jazz network (you can download it from 'http://konect.uni-koblenz.de/networks/arenas-jazz'). The representations are to be obtained the following way. 

1. Let $Z_u$ and $Z_v$ are the representations of nodes $u$ and $v$. At each step of stochastic gradient descent (SGD) you should randomly select a pair of nodes and minimize the loss function - 

   $(Z_u^T Z_v - A_{u,v})^2$
   
2. Obtain another representation of the nodes in the network using the same procedure as in 1 but this time with the loss function as - 

   $(Z_u^T Z_v - A_{u,v})^2 + (Z_u^T Z_v - A_{u,v}^2)^2$
   
3. From these two representations obtain the 5-nearest neighbors of node '0'. The distance between two nodes can be measured as the euclidean distance between the representations of the two nodes.


  
Hints: Calculate the gradient for the loss function and update the representaion vectors using SGD. You can keep the learning rate as 0.001 and the number of iterations as 5000

In [49]:
# constants given by the task description
alpha = 0.001
nIter = 5000

#select a dimension for the representation vector (we chose it randomly)
dim = 13

In [50]:
'''
    We deleted the first line in the file to be able to use the read_edgelist function! (this altered file is also uploaded)
'''
import networkx as nx
G = nx.read_edgelist('out.arenas-jazz')

In [51]:
import numpy
from numpy import random

In [52]:
'''
    Assuming A in the loss function is the adjacency matrix (and not the probabilistic adjacency matrix 
                                        - but this should not make a difference if the graph edges are not weighted)
'''
nodes = list(G.nodes())
A = nx.adjacency_matrix(G)

In [53]:
# initialize the representation between 0 and 1
# Matrix Init of dim 
Init = numpy.random.rand(dim,len(nodes))
Init

array([[0.21911716, 0.08270762, 0.15522897, ..., 0.1111309 , 0.28507101,
        0.76910218],
       [0.25524113, 0.1469989 , 0.6049717 , ..., 0.47271848, 0.87994196,
        0.01706721],
       [0.62659786, 0.96779089, 0.60998802, ..., 0.18619707, 0.81079549,
        0.32899819],
       ...,
       [0.62206829, 0.74814537, 0.98118446, ..., 0.04814932, 0.1389422 ,
        0.80750791],
       [0.14230079, 0.59667203, 0.27614225, ..., 0.45329766, 0.84831743,
        0.34037373],
       [0.14505115, 0.05482138, 0.42995614, ..., 0.73221816, 0.56660156,
        0.4528543 ]])

### 1.

In [54]:
# loss function
def loss(Z,A):
    loss = 0
    for u in range(numpy.shape(A)[0]):
        for v in range(numpy.shape(A)[1]):
            loss += numpy.absolute(  numpy.dot(numpy.transpose(Z[:,u]), Z[:,v])  - A[u,v] )**2
    return loss  #/numpy.shape(A)[0]

In [55]:
# get the representation with loss function given in 1.

# use the initialized matrix
Z_1 = Init.copy() #numpy.random.rand(dim,len(nodes))

for i in range(5000):
    # randomly select two nodes u,v with u!=v
    index_u = random.choice(range(len(nodes)))
    index_v = random.choice(range(len(nodes)))
    
    while (index_v==index_u):
        index_v = random.choice(range(len(nodes)))
    
    # minimize the loss function: $(Z_u^T Z_v - A_{u,v})^2$
    Z_u = Z_1[:,index_u]
    Z_v = Z_1[:,index_v]
    if (i>4995):
        print('current Loss: ', loss(Z_1,A))
    # subtract the gradient from (Z_u,Z_v) to update (Z_u,Z_v)
    Z_1[:,index_u] = Z_u - alpha*numpy.dot( 2*(  numpy.dot(numpy.transpose(Z_u), Z_v)  - A[index_u,index_v] ), Z_v)
    Z_1[:,index_v] = Z_v - alpha*numpy.dot( 2*(  numpy.dot(numpy.transpose(Z_u), Z_v)  - A[index_u,index_v] ), Z_u)
    #print(Z[:,5])

current Loss:  146799.6731476512
current Loss:  146787.75703804582
current Loss:  146774.0183513727
current Loss:  146749.80920943775


### 2.

In [57]:
# do the same as in 1. just with the different loss function
def loss2(Z,A):
    loss = 0
    for u in range(numpy.shape(A)[0]):
        for v in range(numpy.shape(A)[1]):
            loss += (  numpy.dot(numpy.transpose(Z[:,u]), Z[:,v])  - A[u,v] )**2 + (  numpy.dot(numpy.transpose(Z[:,u]), Z[:,v])  - A[u,v]**2 )**2
    return loss

In [58]:
# get the representations with loss function given in 2.

# use the initialized matrix
Z_2 = Init.copy()

for i in range(5000):
    # randomly select two nodes u,v with u!=v
    index_u = random.choice(range(len(nodes)))
    index_v = random.choice(range(len(nodes)))
    while (index_v==index_u):
        index_v = random.choice(range(len(nodes)))    
    
    # minimize the loss function: $(Z_u^T Z_v - A_{u,v})^2 + (Z_u^T Z_v - A_{u,v}^2)^2$
    Z_u = Z_2[:,index_u]
    Z_v = Z_2[:,index_v]
    if (i>4995):
        print('current Loss: ', loss2(Z_2,A), ' in comparison to loss1: ', loss(Z_2,A))
    # subtract the gradient from (Z_u,Z_v) to update (Z_u,Z_v)
    Z_2[:,index_u] = Z_u - alpha*numpy.dot( 2*(  numpy.dot( 2*numpy.transpose(Z_u), Z_v)  - A[index_u,index_v] - A[index_u,index_v]**2 ), Z_v)
    Z_2[:,index_v] = Z_v - alpha*numpy.dot( 2*(  numpy.dot( 2*numpy.transpose(Z_u), Z_v)  - A[index_u,index_v] - A[index_u,index_v]**2 ), Z_u)
    #print(Z[:,5])

current Loss:  156720.65690572115  in comparison to loss1:  78360.32845286057
current Loss:  156705.73826729527  in comparison to loss1:  78352.86913364763
current Loss:  156668.84679350207  in comparison to loss1:  78334.42339675104
current Loss:  156610.4632986081  in comparison to loss1:  78305.23164930406


### 3.

In [59]:
# 3. From these two representations obtain the 5-nearest neighbors of node '0'.
'''
There is no node '0' (as can be seen in the edgelist file), using the node '1' instead!
node '1' has index 0 in the nodes-list
'''
nodes[0]

'1'

In [60]:
# https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy
def distance(a,b):
    return numpy.linalg.norm(a-b)

In [61]:
''' 
    Obtain the five nearest neigbors to node '1' in representation 1
'''
Z_0 = Z_1[:,0] 

# https://stackoverflow.com/questions/36885418/sort-symmetric-numpy-2d-array-by-function-norm
# https://stackoverflow.com/questions/26333005/numpy-subtract-every-row-of-matrix-by-vector
indexlist = numpy.argsort(numpy.linalg.norm(numpy.transpose(numpy.transpose(Z_1)-Z_0),axis=0))

for i in range(6):
    print('closeness rank ', i, ' is node ', nodes[indexlist[i]], ' with distance ', distance(Z_0, Z_1[:,indexlist[i] ]) )

closeness rank  0  is node  1  with distance  0.0
closeness rank  1  is node  170  with distance  0.8037621350145394
closeness rank  2  is node  47  with distance  0.8305363302669926
closeness rank  3  is node  163  with distance  0.9124844424841013
closeness rank  4  is node  141  with distance  0.9983593954402391
closeness rank  5  is node  15  with distance  1.009244024118553


In [62]:
''' 
    Obtain the five nearest neigbors to node '1' in representation 2
'''
Z_0 = Z_2[:,0] 

# https://stackoverflow.com/questions/36885418/sort-symmetric-numpy-2d-array-by-function-norm
# https://stackoverflow.com/questions/26333005/numpy-subtract-every-row-of-matrix-by-vector
indexlist = numpy.argsort(numpy.linalg.norm(numpy.transpose(numpy.transpose(Z_2)-Z_0),axis=0))

for i in range(6):
    print('closeness rank ', i, ' is node ', nodes[indexlist[i]], ' with distance ', distance(Z_0, Z_2[:,indexlist[i] ]) )

closeness rank  0  is node  1  with distance  0.0
closeness rank  1  is node  47  with distance  0.7912126860652018
closeness rank  2  is node  170  with distance  0.8466591579492582
closeness rank  3  is node  163  with distance  0.9345378874652982
closeness rank  4  is node  194  with distance  0.9581871481941598
closeness rank  5  is node  42  with distance  0.9832523631761682
